In [1]:
import pandas as pd
import numpy as np
import openai
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from ast import literal_eval
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
label_names = [
    'ARBEIT.', 'BILDUNG.', 'ENERGIE.', 'FREIZEIT.', 'GESCHICHTE.',
    'GESUNDHEIT.', 'GEWERBE.', 'INDUSTRIE.', 'JUSTIZ.', 'KULTUR.',
    'LANDWIRTSCHAFT.', 'MEDIEN UND KOMMUNIKATION.', 'MENSCH.', 'MILITÄR.',
    'PORTRÄT.', 'RELIGION.', 'SENSATION.', 'SOZIALES.',
    'STAAT UND POLITIK.', 'STRAFTAT.', 'TECHNIK.', 'UMWELT.', 'VERKEHR.',
    'VOLKSKULTUR.', 'WIRTSCHAFT.', 'WISSENSCHAFT.'
]
len(label_names)

26

In [91]:
# df_full = pd.read_excel('../Datenlieferung FARO/Thes-Term_hierarchisch_Kopie.xlsx')
# df_full.groupby('Term - Level 1').aggregate({'Term - Level 2': (lambda x: ','.join([i.lower() for i in x]))})

In [3]:

df_sub = pd.read_excel('../Datenlieferung FARO/Tabelle_Level1und2.xlsx', sheet_name=1).loc[:0, :].set_index('Bezeichnung')
markdown_table = df_sub.transpose()
#markdown_table

In [4]:
# Load and preprocess dataset
df = pd.read_pickle('../Daten/FINAL_faro_cleaned_balanced.pkl')#.sample(10000)   

In [5]:
df.shape

(32799, 30)

In [6]:
# Feature list
X = df['Web-Text']
# Label list 
y = df['Categories'].tolist()  

In [7]:
# First split: Separate out the test set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Second split: Separate the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, shuffle=True)  # 0.25 x 0.8 = 0.2


# Convert to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [8]:
X_train.shape

(19679,)

In [9]:
y_train.shape

(19679, 26)

In [10]:
X_test.shape

(6560,)

In [11]:
y_test.shape

(6560, 26)

## Helperfunctions

In [12]:
import re

def match_and_encode_tags(output, real_tags):
    real_tags = [tag.replace('.', '') for tag in real_tags]
    
    # Define the regex pattern to capture tags with and without trailing dots
    tag_pattern = re.compile(r'\b(' + '|'.join(real_tags) + r')\.?\b', re.IGNORECASE)
    
    # Extract tags from the output
    matched_tags = tag_pattern.findall(output)
    
    # Normalize matched tags by converting them to uppercase and removing trailing dots
    matched_tags = [tag.upper().replace('.', '') for tag in matched_tags]

    # Check if matched_tags are valid, otherwise set all values to zero
    if not matched_tags or any(tag not in real_tags for tag in matched_tags):
        one_hot_encoded = [0] * len(real_tags)
    else:
        # Initialize a one-hot encoded list
        one_hot_encoded = [1 if tag in matched_tags else 0 for tag in real_tags]

    return one_hot_encoded

In [13]:
def visualisation_encodings(response, test_text, df_text):
    
    # The regex pattern
    pattern = r"^\['[A-ZÄÖÜẞ\s]+\.'(\s*,\s*'[A-ZÄÖÜẞ\s]+\.')?\]$"

    print(f'Text: {test_text}')
    # Check each string against the regex
    if re.match(pattern, response.choices[0].message.content):
        print(f"Output Tags (correct format): {response.choices[0].message.content}")
    else:
        print(f"Output Tags (incorrect format): {response.choices[0].message.content}")
        
    # Finding indexes where the value is 1
    print(f'Real Tags: {df_text.columns[df_text.loc[0] == 1].tolist()}')
    print('')
    

In [14]:
def hamming_accuracy(actuals, predictions):
    return (actuals == predictions).mean()

# Chat GPT

In [15]:
from openai import AzureOpenAI 
from getpass import getpass

api_base = getpass("AZURE_OPENAI_ENDPOINT")
api_key= getpass("AZURE_OPENAI_API_KEY")

AZURE_OPENAI_ENDPOINT········
AZURE_OPENAI_API_KEY········


In [16]:
#deployment_name = 'gpt-35-turbo' 
deployment_name = 'gpt-4-preview'
api_version = '2023-12-01-preview' # this might change in the future
client = AzureOpenAI( api_key=api_key, api_version=api_version, base_url=f"{api_base}/openai/deployments/{deployment_name}" )

In [17]:
from tqdm.notebook import tqdm
prompt_accuracy = []
prompt_targets = []
prompt_outputs = []
prompt_text = []

import time
start = time.time()

counter = 0
for test_text in tqdm(X_test):
    
    prompt = f"""
        Angesichts der folgenden beschreibenden Inhalte aus verschiedenen Videos, klassifizieren Sie bitte jeden Ausschnitt, indem Sie bis zu zwei relevante Tags aus der bereitgestellten Tag-Liste zuweisen. Wählen Sie ein Tag, wenn es eindeutig am relevantesten ist, oder zwei Tags, wenn zwei Themen gleichermaßen prominent sind. Die Tags sollten die Hauptthemen des Textes genau widerspiegeln und zum Portfolio eines öffentlich-rechtlichen Senders passen.

        Die Tags und ihre zugehörigen Schlüsselwörter sind in der folgenden Tabelle beschrieben:

        | Tag | Schlüsselwörter |
        |:----|:----------------|
        | ARBEIT. | Arbeit, Arbeitnehmer, Arbeitsbedingung, Arbeitsrecht, Arbeitskonflikt, Arbeitslosigkeit, Arbeitsmarkt, Arbeitsvermittlung, Beruf, Einkommen, Gastarbeiter, Heimarbeit, Kinderarbeit, Karriere, Pensionierung, Schwarzarbeit, Sozialpartnerschaft |
        | BILDUNG. | Berufsbildung, Bildung, Eltern, Erwachsenenbildung, Erziehung, Hochschule, Höhere Fachschule, Fachmittelschule, Gymnasium, Kindergarten, Lehrer, Heil- und Sonderpädagogik, Primarschule, Privatschule, Schule, Schüler, Schulinhalt, Schulreform, Sekundarschule, Stipendium, Sportschule, Studierende |
        | ENERGIE. | Atomenergie, Brennstoff, Elektrizität, Energie, Energieverbrauch, Energietransport, Energiespeicher, Energiepolitik, Erneuerbare Energie, Fossile Energie, Kraftwerk, Staudamm |
        | FREIZEIT. | Alpinismus, Badeanstalt, Breitensport, Denksport, Ferien, Freizeit, Freizeitgarten, Freizeittanz, Geselligkeit, Glücksspiel, Hobby, Jahrmarkt, Nachtleben, Spielen, Sportanlage, Sportgerät, Tourismus, Verein, Wandern |
        | GESCHICHTE. | Geschichte, Geschichte-Früheste, Geschichte-Alte, Geschichte-Mittelalter, Geschichte-Neuere, Geschichte-Neueste |
        | GESUNDHEIT. | Alternativmedizin, Arzt, Drogenpolitik, Ernährung, Gesundheit, Gesundheitswesen, Hygiene, Krankenversicherung, Krankheit, Kur, Medikament, Medizin, Medizinische Forschung, Patientenrecht, Psychiatrie, Rettungswesen, Schwangerschaft, Sucht, Zahnmedizin |
        | GEWERBE. | Autogewerbe, Baugewerbe, Detailhandel, Dienstleistung, Gartenbau, Gastgewerbe, Getränk, Gewerbe, Handwerk, Haustechnik, Immobilienhandel, Kochen, Lebensmittelbranche, Logistik, Möbelbranche, Reiseunternehmen, Sicherheitsbranche |
        | INDUSTRIE. | Autoindustrie, Bergbau, Chemieindustrie, Computerindustrie, Druckindustrie, Flugzeugindustrie, Industrie, Industriestandort, Maschinenindustrie, Metallindustrie, Papierindustrie, Pharmaindustrie, Rohstoff, Rüstungsindustrie, Schiffsbau, Schuhindustrie, Sportartikelindustrie, Tabakindustrie, Textilindustrie, Uhrenindustrie, Unterhaltungselektronik |
        | JUSTIZ. | Bürgerrecht, Eigentumsrecht, Gericht, Gesetz, Internationales Recht, Justiz, Meinungsfreiheit, Menschenrecht, Ombudsstelle, Persönlichkeitsschutz, Personendokument, Strafrecht, Strafverfahren, Strafvollzug, Strafvollzugsanstalt, Urheberrecht, Wettbewerbsrecht, Zivilrecht |
        | KULTUR. | Architektur, Ausstellung, Denkmal, Design, Film, Fotografie, Gedächtnisinstitution, Kino, Kultur, Kulturpolitik, Kulturförderung, Kulturzentrum, Kunst, Literatur, Mode, Musik, Sprache und Schrift, Tanz, Theater, Unterhaltungsbranche |
        | LANDWIRTSCHAFT. | Agrarpolitik, Berglandwirtschaft, Biologische Landwirtschaft, Fischerei, Fleischproduktion, Forstwirtschaft, Jagd, Landwirtschaft, Landwirtschaftsprodukt, Milch, Nutzpflanze, Tierhaltung, Veterinärmedizin, Weinbau |
        | MEDIEN UND KOMMUNIKATION. | Fernsehen, Internet, Journalismus, Medien, Medien und Kommunikation, Medien- und Multimedia-Unternehmen, Medienpolitik, Nachrichtenagentur, Post, Presse, Radio, Satellitenkommunikation, Telekommunikation, Verlag, Video, Zensur |
        | MENSCH. | Alltag, Alter, Asylpolitik, Ausländer, Auslandschweizer, Behinderung, Bevölkerung, Diskriminierung, Ehe, Ethnie, Familie, Flüchtling, Frau, Frauenaktivität, Gesellschaftsform, Gesellschaftsfragen, Grenzgänger, Haushalt, Jugend, Kind, Konkubinat, Liebe, Mann, Mensch, Migration, Minderheit, Randständige, Seelisches, Sexualität, Sterben, Verhalten, Wohnen |
        | MILITÄR. | Internationales Militärbündnis, Kriegsvorsorge, Militär, Militäranlage, Militärausgaben, Militärdienst, Militärjustiz, Militärschau, Rüstung, Schweizergarde, Truppengattung, Truppenstationierung, Waffe, Militär, Waffe, Privat, Zivildienst, Zivilschutz |
        | PORTRÄT. | Porträt Geografie, Porträt Körperschaften, Porträt Personen, Porträt Thema |
        | RELIGION. | Christentum, Christliche Kirche, Glaube, Gott, Mission, Nichtchristliche Religion, Ökumene, Religiöse Schriften, Religiöser Fundamentalismus, Religiöse Sondergemeinschaft, Religion, Spiritualität |
        | SENSATION. | Affäre, Auszeichnung, Brand, Erfindung, Expedition, Explosion, Kuriosa, Prominenz, Sensation, Showbusiness, Unfall |
        | SOZIALES. | Altersvorsorge, Armut, Demoskopie, Feier, Heim, Hilfswerk, Humanitäre Hilfe, Lebensstandard, Reichtum, Sozialarbeit, Soziale Bewegung, Soziales, Sozialhilfe, Sozialpolitik, Sozialversicherung, Protest, Vormundschaft, Zukunft |
        | STAAT UND POLITIK. | Abstimmung, Amtsausübung, Aussenpolitik, Bund, Europäische Integration, Feuerwehr, Finanzen, Frieden, Gemeinde, Gewaltentrennung, Gleichstellung, Grenze, Heimat, Ideologie, Initiative, Innenpolitik, Internationale Organisation, Kanton, Krieg, Landesausstellung, Lobbyismus, Mass, Neutralität, Ost-West, Parlament, Politische Gruppierung, Polizei, Referendum, Reformprozess, Regierung, Regionalismus, Siedlungspolitik, Staat und Politik, Staatsbetrieb, Staatsfeier, Staatsform, Staatssicherheit, Staatszugehörigkeit, Steuer, Subvention, Umsturz, Unruhen, Verfassung, Verwaltung, Wahl, Wiedergutmachung, Weltausstellung, Widerstandsbewegung, Zoll, Zwischenstaatlicher Konflikt |
        | STRAFTAT. | Bombenalarm, Doping, Drogendelikt, Ehrendelikt, Eigentumsdelikt, Fälschung, Gewalt gegen Personen, Korruption, Kriminelle Vereinigung, Kriegsverbrechen, Politische Verbrechen, Sachbeschädigung, Sexualdelikt, Straftat, Spionage, Tötungsdelikt, Verbrechensverhütung, Verkehrsdelikt, Wirtschaftskriminalität |
        | TECHNIK. | Automatisierung, Computertechnik, Digitalisierung, Elektrotechnik, Präzisionsinstrument, Raumfahrttechnik, Software, Technik, Überwachungstechnik, Werkstoff, Werkzeug |
        | UMWELT. | Abfall, Atom, Atommüll, Boden, Gewässer, Giftstoff, Heimatschutz, Klima, Lärm, Landschaft, Luft, Naturkatastrophe, Raumplanung, Recycling, Schädlingsbekämpfung, Stadt, Tier, Umwelt, Umweltschutz, Wald, Wasser, Wetter |
        | VERKEHR. | Alpentransversale, Auto, Autobahn, Bahn, Bahnanlage, Bus, Bergbahn, Brücke, E-Fahrzeug, Fahrplan, Fahrpreis, Fahrrad, Fluggesellschaft, Flughafen, Flugzeug, Fussgänger, Güterverkehr, Historische Verkehrsmittel, Luftverkehr, Motorrad, Nahverkehr, Öffentlicher Verkehr, Parken, Pendlen, Raumfahrt, Schifffahrt, Strassenbau und Unterhalt, Strassenverkehr, Tram, Tunnel, Verkehr, Verkehrsverband, Verkehrsverbund, Verkehrspolitik |
        | VOLKSKULTUR. | Brauchtum, Brauchtumssport, Feiertag, Jahreszeit, Zunft |
        | WIRTSCHAFT. | Arbeitgeber, Aussenwirtschaft, Betriebsänderung, Bodenmarkt, Finanzbranche, Geld, Gold, Handel, Investition, Kapitalmarkt, Kartell, Konjunktur, Konkurs, Konsum, Kredit, Preisniveau, Selbstständigkeit, Sponsoring, Unternehmen, Unternehmenskonzentration, Versicherungsbranche, Währung, Weltwirtschaft, Werbung, Wirtschaft, Wirtschaftspolitik, Wirtschaftsstatistik, Zentralbank |
        | WISSENSCHAFT. | Archäologie, Astronomie, Biologie, Chemie, Ethnologie, Geisteswissenschaft, Genetik, Geografie, Geologie, Geschichtswissenschaft, Grenzwissenschaft, Informatik, Kartografie, Literaturwissenschaft, Mathematik, Meteorologie, Naturwissenschaft, Paläontologie, Philosophie, Physik, Politologie, Psychologie, Rechtswissenschaft, Religionswissenschaft, Soziologie, Sportwissenschaft, Sprachwissenschaft, Theologie, Wirtschaftswissenschaft, Wissenschaft |

        Text zum Klassifizieren: '{test_text}'

        Ihre Aufgabe ist es, den Ausschnitt zu lesen und die am besten passenden Tags zuzuweisen, die den Inhalt am besten widerspiegeln, unter Berücksichtigung ihrer Relevanz für den öffentlich-rechtlichen Rundfunk und die Informationen in der Tabelle. 
        Geben Sie nur die Tags als Python-Liste aus.
    """
    
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "Sie sind ein hilfreicher Assistent, der dem Redaktionsteam des öffentlich-rechtlichen Rundfunks hilft, zentrale Themen aus beschreibenden Videotexten zu extrahieren."},
                {"role": "user", "content": prompt}
            ], timeout=5
        )
    except Exception as e:
        continue
        print(f"API Error: {test_text}, Error: {e}")
    
    
    # Transfer ChatGPT output to One-Hot encoded list
    try:
        output_tags = response.choices[0].message.content
        one_hot_encoded_tags = match_and_encode_tags(output_tags, label_names)
        df_text = df[df['Web-Text'] == test_text].reset_index(drop=True)
        
        prompt_outputs.append(one_hot_encoded_tags)
        prompt_targets.append(df_text['Categories'][0]) 
        prompt_text.append(test_text)
    
    except Exception as e:
        print(f"Error processing text:{e}")
        print(response.choices[0].message.content) 
        
        prompt_outputs.append(np.nan)
        prompt_targets.append(np.nan) 
        prompt_text.append(test_text)
        prompt_accuracy.append(np.nan)
        continue
    
    
    
    
    # Compute Accuracy on the fly
    match_percent = sum(i==j for i,j in zip(df_text['Categories'][0],one_hot_encoded_tags))/len(one_hot_encoded_tags)
    prompt_accuracy.append(match_percent)
    
    # Print some of the Answers
    if counter % 15 == 0:
        # Show the Answer
        visualisation_encodings(response, test_text, df_text)
    counter += 1
    

end = time.time()
print(end - start)

  0%|          | 0/6560 [00:00<?, ?it/s]

Text: Türkei stimmt für Schwedens Nato-Beitritt
Das türkische Parlament hat den Beitritt Schwedens in die Nato gebilligt. Nun muss Präsident Erdogan das sogenannte Beitrittsprotokoll noch unterschreiben, was als so gut wie sicher gilt.
Output Tags (incorrect format): ```python
["STAAT UND POLITIK", "MILITÄ
Real Tags: ['MILITÄR.']

Text: Wie funktionieren Wetten?
In einem Beitrag werden die diversen möglichen Wetten aufgezeigt, die auf jedes Spiel eingegangen werden können. Zudem wird klar, dass man auf irgendein Spiel setzen kann  ganz egal, wo es stattfindet. Roger Hegi betont, dass Wetten auf die manipulierten Schweizer Spiele vor allem in Asien platziert wurden. Markus Stähli begründet, wieso alle Thuner Spieler eine Erklärung unterschreiben mussten, dass sie nicht betrogen haben.
Output Tags (incorrect format): ["SPORT", "JUSTIZ"]
Real Tags: ['FREIZEIT.', 'MEDIEN UND KOMMUNIKATION.', 'STRAFTAT.']

Text: Langlauf und Tourenskifahren sind die besten Kalorien-Killer
Die Hauptmotivati

Text: Brüssel will ökologischen Anbau fördern
Die EU hat ihr Agrar-Konzept für die Zukunft vorgestellt. Die Landwirtschaftspolitik soll grüner und gerechter werden.
Output Tags (incorrect format): ```python
["LANDWIRTSCHAFT.", "UMWELT
Real Tags: ['LANDWIRTSCHAFT.', 'STAAT UND POLITIK.']

Text: Tiramisu: Bauchweh nach dem Verzehr?

Output Tags (incorrect format): ["GESUNDHEIT."]
Real Tags: ['GESUNDHEIT.', 'GEWERBE.']

Text: SRF Börse vom 13.11.2018
Während sich andere Pharmakonzerne zurückziehen, setzt Biogen auf Alzheimer-Forschung – trotz hohem Risiko. 90 Prozent der Projekte scheitern. Die Gesellschaft altere, Nerven-Krankheiten würden künftig für Gesundheitsbehörden weltweit oberste Priorität, sagt Biogen-Chef Michel Vounatsos. SMI +0.3%
Output Tags (incorrect format): ```python
["WIRTSCHAFT.", "GESUNDHEIT."]

Real Tags: ['GESUNDHEIT.', 'INDUSTRIE.', 'WISSENSCHAFT.']

Text: Proteste in Serbien gegen Corona-Schutzmassnahmen
Gegen die von Präsident Vucic verordneten Corona-Schutzmassn

Text: Grünes Licht für die erste Covid-19-Impfung in der Schweiz
Swissmedic hat das Vakzin von Biontech/Pfizer als ersten Impfstoff für die Schweiz gegen das neue Coronavirus zugelassen. Der Impfstoff soll rasch zum Einsatz kommen. Einschätzungen von SRF-Wissenschaftsredaktor Thomas Häusler.
Output Tags (incorrect format): ["GESUNDHEIT.", "WISSENSCHAFT."]
Real Tags: ['GESUNDHEIT.']

Text: Ein Dorf kämpft gegen einen Fassreiniger
Einem Dorf stinkt es gewaltig. Seit Jahren verpestet ein Betrieb, der Industriefässer reinigt, die Luft. Bei Westwind weht ekelhafter Gestank in die Stuben. Alle Proteste nützen nichts. Die Behörden haben zwar Sanierungsmassnahmen verfügt, doch das sei zu wenig, klagen die Dorfbewohner.
Output Tags (incorrect format): ```python
[UMWELT, STAAT UND POLITIK]

Real Tags: ['INDUSTRIE.', 'UMWELT.']

Text: Japan als Lichtblick für Schweizer Firmen
Die Schweizer Exporte nach Japan boomen. Die Zuwachszahl beträgt satte 37 Prozent. Im Gegensatz dazu sind die Exporte in e

Text: Streit um Fallpauschalen
Seit Anfang Jahr rechnen Spitäler ihre Leistungen anhand sogenannter Fallpauschalen ab. Schon vor der Einführung des neuen Systems äusserten Spitaldirektoren in «ECO» Befürchtungen: Das neue System bedrohe sie in ihrer Existenz, weil anstehende Investitionen – etwa im Gebäudebereich – nicht gedeckt seien. Jetzt haben sich die Bedenken zu einem offenen Konflikt zwischen Spitälern und Krankenversicherern gewandelt.
Output Tags (incorrect format): ```python
["GESUNDHEIT.", "WIRTSCHAFT."]

Real Tags: ['GESUNDHEIT.']

Text: «Die Idee»: Digitaler Blindenstock
Verletzungen im öffentlichen Raum gehören für Blinde zum Alltag. Elektronische Technik warnt nun vor Hindernissen – für Blinde ein Stück Freiheit.
Output Tags (incorrect format): ```python
['TECHNIK.', 'GESUNDHEIT.']

Real Tags: ['MENSCH.', 'TECHNIK.']

Text: Die Akteure hinter Gerigate

Output Tags (incorrect format): ```python
["SENSATION", "JUSTIZ"]
```
Real Tags: ['MEDIEN UND KOMMUNIKATION.', 'SENSATIO

...

Text: Schweiz aktuell - Thema Kinder
Im Rahmen der Kinderwoche des Schweizer Fernsehens waren Claudia Weber und Sabine Dahinden heute in Baar. Dort besuchten sie das heilpädagogische Schul- und Beratungszentrum Sonneberg, wo blinde und sehbehinderte Kinder unterrichtet werden. Ausserdem meldet sich Sabine Dahinden aus Amriswil im Kanton Thurgau, der nächsten Station unserer Reise von Schulstube zu Schulstube.
Output Tags (incorrect format): ```python
["BILDUNG.", "MENSCH."]
```
Real Tags: ['BILDUNG.', 'MENSCH.']

Text: Anti-Frontex Komitee prangert Gewalt an EU-Aussengrenze an
Bundesrat und Parlament möchten die Frontex stärker unterstützen. Dagegen hat eine Allianz aus Linken, Grünen und Menschenrechtsorganisationen das Referendum ergriffen. Die Frontex sei an Menschenrechtsverletzungen beteiligt.
Output Tags (incorrect format): ```python
[STAAT UND POLITIK, MENSCH]
```
Real Tags: ['JUSTIZ.', 'MENSCH.', 'STAAT UND POLITIK.']

Text: SRF Börse vom 08.11.2017
Der weltweit grösste Schokol

Text: Trinkfreude auch ohne Promille: Alkoholfreie Biere im Vergleich
Ein alkoholfreies Bier im Sortiment, das ist für viele Brauereien mittlerweile ein Muss. Welches Gebräu überzeugt bei Schaum, Aromatik und Geschmack? Und wie schlagen sich die Schweizer Biere gegen die grosse Konkurrenz aus Deutschland? Antworten in der «Kassensturz»-Degustation.
Output Tags (incorrect format): ```python
["FREIZEIT.", "WIRTSCHAFT."]

Real Tags: ['GEWERBE.']

Text: Millionen-Geschenk für Nestlé
Im Freiburgischen Romont wurde heute die dritte Nespresso-Fabrik in der Schweiz feierlich eingeweiht. Die Region erhofft sich davon einen wirtschaftlichen Aufschwung mit vielen neuen Arbeitsplätzen. Zu reden gibt jedoch, dass Nestlé in Romont während 10 Jahren keine Steuern bezahlt.
Output Tags (incorrect format): ```python
["WIRTSCHAFT", "ARBEIT"]
```
Real Tags: ['STAAT UND POLITIK.', 'WIRTSCHAFT.']

Text: Student tötet mindestens 14 Menschen an Universität in Prag
An der Karls-Universität in der tschechischen

Text: Allein in der Sperrzone: Doris Dörries «Grüsse aus Fukushima»
Die deutsche Filmregisseurin Doris Dörrie setzt den Opfern von Fukushima ein filmisches Denkmal. Im Katastrophengebiet erzählt sie die Geschichte von Marie, die vor dem Verlust ihrer grossen Liebe nach Japan flieht, um Überlebenden der Tsunami-Katastrophe zu helfen. Doch die junge Deutsche bleibt in ihrer Verzweiflung gefangen. Erst bei einer älteren, störrischen Geisha, die auf eigene Faust in ihr zerstörtes Haus in der Sperrzone zurückzieht, findet sie Orientierung. Zwei unterschiedliche Frauen, die lernen müssen, sich von ihren Schuldgefühlen und der Last ihrer Erinnerungen zu befreien.
Output Tags (incorrect format): ```python
["KULTUR", "UMWELT"]
```
Real Tags: ['KULTUR.', 'SENSATION.']

Text: 20 Jahre nach dem Holocaust-Vergleich
Vor 20 Jahren einigten sich Schweizerische Banken mit jüdischen Sammelklägern in den USA auf eine Entschädigungszahlung – für die Kooperation mit dem Nazi-Regime. Aufgearbeitet sind dami

Text: Sängerin Sina – Raubkatze UND Fluchtier

Output Tags (incorrect format): ```python
["KULTUR", "PORTRÄT"]
```
Real Tags: ['KULTUR.']

Text: Hüttenbau vom Wohnzimmer aus
Neu kann man die Hüttenbauer im Internet selbst spielen und online ein Haus im Wald bauen. Schweiz aktuell wollte wissen, wie so ein Spiel entwickelt wird und ob die Hüttenbauer mit sich selbst als Spielfigur zufrieden sind.
Output Tags (incorrect format): ```python
["FREIZEIT.", "MEDIEN UND K
Real Tags: ['FREIZEIT.', 'MEDIEN UND KOMMUNIKATION.', 'UMWELT.']

Text: Gute Prognosen für Schweizer Wirtschaft
Die Schweizer Wirtschaft - letztes Jahr noch gut ein Prozent gewachsen - wird dieses Jahr in den Augen vieler Prognostiker um über zwei Prozent zulegen. Und auch die Börse brummt. Der Stimmungsbericht von unserer Wirtschaftsredaktion.
Output Tags (incorrect format): ```python
["WIRTSCHAFT."]
```
Real Tags: ['WIRTSCHAFT.']

Text: Die eigentlichen Machthaber Ägyptens
Nach dem Abtreten Mubaraks übernimmt nun der Militä

Text: Sportzentrum in Magglingen feiert 75-Jahr-Jubiläum
1944 ist oberhalb von Biel der Grundstein gelegt worden für das Nationale Sportzentrum Magglingen. Anlass genug für eine 75-Jahr-Feier mit vielen Prominenten - und für einen Rundgang mit Kugelstoss-Weltmeister Werner Günthör.
Output Tags (incorrect format): ["FREIZEIT.", "KULTUR."]
Real Tags: ['GESCHICHTE.', 'SOZIALES.', 'WISSENSCHAFT.']

Text: Hart am Limit
Hintergründe zum Eklat um den kroatischen Popstar Thompson
Output Tags (incorrect format): ```python
["KULTUR", "SENSATION"]
```
Real Tags: ['KULTUR.', 'STAAT UND POLITIK.']

Text: Mit Köpfchen zu mehr Ausdauer?
Die Eidgenössische Hochschule für Sport in Magglingen hat eine neue Studie veröffentlicht. Der Faktor Kopf spielt im Ausdauerbereich eine wichtige Rolle. Mit der richtigen mentalen Einstellung kann man die Ausdauerfähigkeit bis zu 20 Prozent steigern. Ist diese Beharrlichkeit trainierbar?
Output Tags (incorrect format): ```python
["WISSENSCHAFT", "SPORT"]
```
Real Tag

In [18]:
df_results = pd.DataFrame([prompt_outputs, prompt_targets, prompt_text, prompt_accuracy]).transpose()
df_results.columns = ['outputs', 'targets', 'text', 'accuracy']
df_results.to_pickle('../Daten/Final_ChatGPT_Answers.pkl')

In [20]:
prompt_accuracy_ = df_results['accuracy'].dropna().tolist()
np.mean(prompt_accuracy_)

0.9365623635226215

In [21]:
prompt_targets_ = df_results['targets'].dropna().tolist()
prompt_outputs_ = df_results['outputs'].dropna().tolist()

hamming_accuracy(np.array(prompt_targets_),np.array(prompt_outputs_))

0.9365623635226213

In [22]:
from sklearn.metrics import classification_report

report = classification_report(prompt_targets_, prompt_outputs_, target_names=label_names)
print(report)

                           precision    recall  f1-score   support

                  ARBEIT.       0.59      0.56      0.57       381
                 BILDUNG.       0.80      0.76      0.78       320
                 ENERGIE.       0.94      0.68      0.79       349
                FREIZEIT.       0.38      0.46      0.42       424
              GESCHICHTE.       0.57      0.63      0.60       284
              GESUNDHEIT.       0.83      0.63      0.72       715
                 GEWERBE.       0.76      0.15      0.25       518
               INDUSTRIE.       0.72      0.39      0.50       352
                  JUSTIZ.       0.49      0.64      0.56       524
                  KULTUR.       0.54      0.79      0.64       736
          LANDWIRTSCHAFT.       0.87      0.44      0.58       396
MEDIEN UND KOMMUNIKATION.       0.67      0.24      0.35       480
                  MENSCH.       0.63      0.35      0.45       742
                 MILITÄR.       0.76      0.48      0.59     

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Example to convert your list of arrays into a single matrix
actuals_matrix = np.vstack(prompt_targets_)  # Do the same for predictions if necessary

# Assuming 'predictions' is already in a similar format or adjust accordingly
predictions_matrix = np.vstack(prompt_outputs_)

mean_sample_accuracy = []
print("Label-Accuracy:")
for i in range(actuals_matrix.shape[1]):  # Assuming actuals and predictions are numpy arrays
    label_accuracy = accuracy_score(actuals_matrix[:, i], predictions_matrix[:, i])
    mean_sample_accuracy.append(label_accuracy)
    print(f'{label_names[i]}: {round(label_accuracy, 3)}')

Label-Accuracy:
ARBEIT.: 0.951
BILDUNG.: 0.979
ENERGIE.: 0.981
FREIZEIT.: 0.917
GESCHICHTE.: 0.963
GESUNDHEIT.: 0.945
GEWERBE.: 0.929
INDUSTRIE.: 0.959
JUSTIZ.: 0.918
KULTUR.: 0.899
LANDWIRTSCHAFT.: 0.962
MEDIEN UND KOMMUNIKATION.: 0.935
MENSCH.: 0.903
MILITÄR.: 0.964
PORTRÄT.: 0.902
RELIGION.: 0.971
SENSATION.: 0.925
SOZIALES.: 0.909
STAAT UND POLITIK.: 0.857
STRAFTAT.: 0.928
TECHNIK.: 0.956
UMWELT.: 0.939
VERKEHR.: 0.961
VOLKSKULTUR.: 0.96
WIRTSCHAFT.: 0.892
WISSENSCHAFT.: 0.947


In [53]:
(actuals_matrix[:, 0] == predictions_matrix[:, 0]).mean()

0.9512045419671628

In [54]:
pd.read_pickle('../Daten/Final_ChatGPT_Answers.pkl')

,outputs,targets,text,accuracy
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Türkei stimmt für Schwedens Nato-Beitritt\nDas...,0.923077
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",Faules Geschäft mit Bio-Eiern\n,0.961538
2,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Kinder-Spezial – Tag 5 – Bäckerei Voland, Steg...",0.884615
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, ...",Endlich frei - Yvonne Catterfeld will ihr Imag...,0.884615
4,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Gracy wird nicht blind werden\nBeim Spielen mi...,1.0
...,...,...,...,...
6512,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",«Bruno Manser - die Stimme des Regenwaldes»\nD...,0.961538
6513,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",Starkoch: Andreas Caminada in der Weltelite\nN...,0.846154
6514,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Alpabfahrt in Urnäsch: die Highlights\nSechs S...,1.0
6515,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Kein Kriegsende in Sicht in Syrien\nAn eine ra...,0.961538
